In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
import seaborn as sns
from plotting_tools import save_fig

def plot_plositions(row, min_time=None, max_time=None, max_dist=None):
    positions_cm = row.positions[:, :3] * 100
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = np.ones_like(row.seconds, dtype=np.bool)
    if min_time is not None:
        mask_time = (row.seconds > min_time) 
    if max_time is not None:
        mask_time = mask_time & (row.seconds < max_time)
        
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    if max_dist is not None:
        axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')
    plt.tight_layout()
    return fig, axs

def plot_audio(row, mic_idx=0):
    all_frequencies = np.unique(row.frequencies_matrix)
    spec = row.spectrogram[:, mic_idx, :]
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    pcolorfast_custom(ax, row.seconds, all_frequencies, np.abs(spec))
    ax.set_ylabel('frequency [Hz]')
    return fig, ax

# 1. Frequency slice

In [ ]:
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
#exp_name = '2021_04_30_hover';
exp_name = '2021_05_04_flying';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 1.1 positions analysis

In [ ]:
min_time = None
max_time = None
max_dist = None
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time, max_time, max_dist)
    save_fig(fig, f'plots/{exp_name}{row.appendix}_movement.pdf')

# 1.2 plot audio

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    fig, ax = plot_audio(row, mic_idx=mic_idx)

# frequency slices

In [ ]:
from geometry import Context
context = Context.get_crazyflie_setup(dim=2)
context.plot()

In [ ]:
# choose the experiment to analyze
row = df_total.iloc[0] # works well
#row = df_total.iloc[1] # works ok
#row = df_total.iloc[2] # doesn't work

In [ ]:
from calibration import get_calibration_function_median, get_calibration_function_dict

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
if "newbuzzer" in row.appendix:
    calib_function, calib_freq = get_calibration_function_median(
        exp_name="2021_02_25_wall", mic_type="audio_deck", ax=ax
    )
else:
    calib_function, calib_freq = get_calibration_function_median(
        "2021_04_30_stepper", "audio_deck", ax=ax, #fit_one_gain=True 
    )
#else:
#    calib_function, calib_freq = get_calibration_function_median(
#        exp_name="2021_02_23_wall", mic_type="audio_deck", ax=ax
#    )


In [ ]:
# spec_masked, freqs_masked = data_collector.fill_from_row()
from data_collector import DataCollector
from estimators import DistanceEstimator
from inference import get_probability_bayes
from simulation import get_freq_slice_theory
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

EPS = 1e-10
    
starting_distance = 60  # [50, 30, 10]
nominal_distances = [60-i*20 for i in range(3)]
slice_idx = 0

distance_range = [0, 70]
azimuth_deg = WALL_ANGLE_DEG

data_collector = DataCollector(exp_name=exp_name)

fig, ax_distance = plt.subplots()
fig.set_size_inches(10, 5)
ax_distance.set_xlabel("time idx")
ax_distance.set_ylabel("distance [cm]")
ax_distance.axhline(starting_distance, label="starting distance", ls=":")
ax_distance.set_title(f"experiment {row.appendix}")

fig, ax_total = plt.subplots()
fig.set_size_inches(10, 5)
ax_total.set_xlabel("distance [cm]")
ax_total.set_ylabel("probability")
ax_total.set_title(f"experiment {row.appendix}")

freqs_calib = np.linspace(np.min(row.frequencies_matrix), np.max(row.frequencies_matrix), 30)
f_calib_all = calib_function(freqs_calib)

for i in range(row.stft.shape[0]):
    signals_f = row.stft[i]
    frequencies = row.frequencies_matrix[i]

    if i == row.stft.shape[0] - 1:
        sweep_complete = True
    else:
        sweep_complete = data_collector.detect_sweep(signals_f, frequencies)

    if sweep_complete:
        f_slice, freqs, stds, d_slice = data_collector.get_current_frequency_slice(
            verbose=False
        )

        fig, axs = plt.subplots(3, row.stft.shape[1])  # , sharey='row')
        fig.set_size_inches(15, 10)
        fig.suptitle(f"up to {data_collector.latest_time:.1f}s", y=0.9)
        #fig.set_suptitle(f"experiment {row.appendix}")
        
        # extract valid indices
        valid_idx = (
            (freqs >= min(calib_function.x))
            & (freqs <= max(calib_function.x))
            & ((3800 <= freqs) | (freqs <= 3700))
        )
        freqs = freqs[valid_idx]
        f_calib = calib_function(freqs)
        f_slice = f_slice[:, valid_idx]
        d_slice = d_slice[valid_idx]
        
        distance_corr = starting_distance - (d_slice * 1e2)

        f_theo = get_freq_slice_theory(
            freqs, distance_cm=nominal_distances[slice_idx], azimuth_deg=azimuth_deg
        ).T  # n_mics x n_freqs
        f_theo_corr = get_freq_slice_theory(
            freqs, distance_cm=distance_corr, azimuth_deg=azimuth_deg
        ).T

        print("treating new frequency slice")

        distance_estimators = {
            'measured': DistanceEstimator(),
            'theo': DistanceEstimator(),
            'theo_corr': DistanceEstimator(),
        }
        linestyles = {
            'measured': "-",
            'theo': ":",
            'theo_corr': "-."
        }

        for i_mic in range(f_slice.shape[0]):
            axs[0, i_mic].plot(freqs, f_slice[i_mic], label="measured")
            axs[0, i_mic].legend(loc="upper right")

            f_calibrated = f_slice[i_mic] / f_calib[i_mic]

            axs[1, i_mic].plot(freqs, f_calibrated, label="measured", ls="-")
            axs[1, i_mic].plot(freqs, f_theo[i_mic], label="theo", color=f"C6", ls=":")
            axs[1, i_mic].plot(
                freqs, f_theo_corr[i_mic], label="theo corr", color="C7", ls="-."
            )
            axs[0, i_mic].set_title(f"mic{i_mic}")

            # get probability distribution
            distances_bayes, proba, diff_bayes = get_probability_bayes(
                f_calibrated[~np.isnan(f_calibrated)],
                freqs[~np.isnan(f_calibrated)],
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg
            )
            proba = (proba - np.min(proba) + EPS) / (
                np.max(proba) - np.min(proba) + EPS
            )
            
            # get probability distribution with theoretical
            distances_bayes_theo, proba_theo, diff_bayes_theo = get_probability_bayes(
                f_theo[i_mic],
                freqs,
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg
            )
            proba_theo = (proba_theo - np.min(proba_theo) + EPS) / (
                np.max(proba_theo) - np.min(proba_theo) + EPS
            )
            
            distances_bayes_theo_corr, proba_theo_corr, diff_bayes_theo_corr = get_probability_bayes(
                f_theo_corr[i_mic],
                freqs,
                mic_idx=i_mic,
                distance_range=distance_range,
                sigma=1.0,
                azimuth_deg=azimuth_deg
            )
            proba_theo_corr = (proba_theo_corr - np.min(proba_theo_corr) + EPS) / (
                np.max(proba_theo_corr) - np.min(proba_theo_corr) + EPS
            )
            distance_estimators['measured'].add_distribution(diff_bayes * 1e-2, proba, i_mic)
            distance_estimators['theo'].add_distribution(diff_bayes_theo * 1e-2, proba_theo, i_mic)
            distance_estimators['theo_corr'].add_distribution(diff_bayes_theo_corr * 1e-2, proba_theo_corr, i_mic)

            axs[2, i_mic].plot(distances_bayes, proba, label='measured')
            axs[2, i_mic].plot(distances_bayes_theo, proba_theo, label='theo')
            axs[2, i_mic].plot(distances_bayes_theo_corr, proba_theo_corr, label='theo_corr')

        for key, distance_estimator in distance_estimators.items():
            distance_total, proba_total = distance_estimator.get_distance_distribution(
                method="sum", chosen_mics=None, azimuth_deg=azimuth_deg
            )
            ax_total.plot(
                distance_total * 1e2, proba_total, label=f"{key} d={nominal_distances[slice_idx]}cm",
                color=f"C{slice_idx}", ls=linestyles[key]
            )

        axs[1, i_mic].legend()
        axs[2, i_mic].legend()
        ax_distance.plot(distance_corr, label=nominal_distances[slice_idx])

        for i_mic in range(f_calib_all.shape[0]):
            axs[0, i_mic].plot(freqs_calib, f_calib_all[i_mic], label=f"calib", ls=":")
        slice_idx += 1

    data_collector.fill_from_signal(
        signals_f, frequencies, distance=row.positions[i, 1], time=row.seconds[i]
    )

ax_distance.legend(loc="upper right")
ax_total.legend(loc="upper right")

In [ ]:
from dataset_parameters import kwargs_datasets
from simulation import get_df_theory

distances_grid = np.arange(distance_range[0]+10, distance_range[1])

kwargs = kwargs_datasets[exp_name]["audio_deck"]
freqs_theo = np.linspace(kwargs["min_freq"], kwargs["max_freq"], 100)

df_matrix_theo = get_df_theory(freqs_theo, distances_grid, azimuth_deg=azimuth_deg+1, 
                               chosen_mics=[mic_idx])
fig, ax = plt.subplots()
pcolorfast_custom(ax, distances_grid, freqs_theo, np.log10(df_matrix_theo[0]))
ax.set_yticklabels([''])
ax.set_xticklabels([''])

# 2. Distance slice

In [ ]:
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
#exp_name = '2021_04_30_hover';
exp_name = '2021_05_04_linear';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

## 1.1 positions analysis

In [ ]:
min_time = 0
max_time = np.inf
max_dist = 100 # cm
for i, row in df_total.iterrows():
    fig, axs = plot_plositions(row, min_time=None, max_time=None, max_dist=None)
    #save_fig(fig, f'plots/{exp_name}{row.appendix}_movement.pdf')

# 1.2 plot audio

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0
#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    fig, ax = plot_audio(row, mic_idx=mic_idx)
    
    #plt.figure()
    #for time_idx in range(row.frequencies_matrix.shape[0]):
    #    mic_idx = 0
    #    freqs = row.frequencies_matrix[time_idx, :]
    #    response = np.abs(row.spectrogram[:, mic_idx, time_idx])
    #    plt.plot(freqs, response)

In [ ]:
gt_values = {
    '_1': 45,
    '_2': -45,
    '_3': 90,
    '_4': -30,
    '_5': 30,
    '_fast1': -45,
    '_fast2': 45,
    '_fast3': 90,
    '_fast4': 30,
    '_fast5': -30
}

In [ ]:
from inference import get_approach_angle_fft

for i_row, row in df_total.iterrows():
    data_collector = DataCollector(exp_name=exp_name)
    for i in range(row.stft.shape[0]):
        signals_f = row.stft[i]
        frequencies = row.frequencies_matrix[i]
        data_collector.fill_from_signal(
            signals_f, frequencies, distance=row.positions[i, 1], time=row.seconds[i]
        )
    data_collector.to_numeric()
    
    n_mics = len(data_collector.df.mic.unique())
        
    fig, axs = plt.subplots(1, n_mics, sharey=True)
    fig.set_size_inches(15, 5)
    for i_mic, (mic, df_mic) in enumerate(data_collector.df.groupby('mic')):
        axs[i_mic].plot(df_mic.distance[1:]*1e2, df_mic.magnitude[1:], color='C0')
        axs[i_mic].set_title(f'mic{i_mic}')
        axs[i_mic].set_xlabel('distance [cm]')
    axs[0].set_ylabel(f"magnitude{row.appendix}")
        
    counts = data_collector.df.groupby('frequency').time.count() # doesn't matter if we use time or other column
    f_best = counts.index[np.argmax(counts.values)]
    
    fig, axs = plt.subplots(1, n_mics, sharey=True)
    fig.set_size_inches(15, 5)
    for i_mic in range(n_mics):
        d_slice, distances, *_ = data_collector.get_distance_slice(mic=i_mic)#, frequency=f_best)
        #d_slice -= np.mean(d_slice, axis=1)[None, :]
        angles, proba = get_approach_angle_fft(
            d_slice=d_slice[0],
            frequency=np.mean(counts.index),
            relative_distances_cm=distances*1e2,
            reduced=True
        )
        axs[i_mic].set_title(f'mic{i_mic}')
        axs[i_mic].plot(angles, proba, color='C1')
        axs[i_mic].axhline(gt_values[row.appendix])
    axs[0].set_ylabel(f"probability{row.appendix}")